In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [2]:
out='../output/'
inb='../input/flows/'
pref='dam'
inp = inb + pref + '/'

In [3]:
dat=pd.DataFrame(columns=['date','flow','from','hour','to'])
onlyfiles = [f for f in listdir(inp ) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat['flow']=dat['flow'].astype(int)
dat

,date,flow,from,hour,to
0,2016-01-01,160,ZAMZ,0,ZAMC
1,2016-01-01,170,ZIM,0,ZAMZ
2,2016-01-01,160,ZAMZ,1,ZAMC
3,2016-01-01,170,ZIM,1,ZAMZ
4,2016-01-01,160,ZAMZ,2,ZAMC
...,...,...,...,...,...
3770,2021-12-31,62,MOZS,23,RSAN
3771,2021-12-31,67,RSAN,23,BOT
3772,2021-12-31,124,ZAMZ,23,ZAMC
3773,2021-12-31,124,ZIM,23,ZAMZ


In [4]:
dam = pd.read_csv(out + pref + '_hourly.csv')
dam=dam.set_index([pd.DatetimeIndex(dam['date']),'hour'])
dam=dam.drop(['date'],1)
dam

price  volume  year  month  day  day_of_week      value  \
date       hour                                                             
2016-01-01 0      40.10   477.5  2016      1    1            4  19147.750   
           1      40.08   486.6  2016      1    1            4  19502.928   
           2      40.06   495.6  2016      1    1            4  19853.736   
           3      40.08   496.6  2016      1    1            4  19903.728   
           4      40.07   502.9  2016      1    1            4  20151.203   
...                 ...     ...   ...    ...  ...          ...        ...   
2021-11-30 19    319.50     5.0  2021     11   30            1   1597.500   
           20    319.50     5.0  2021     11   30            1   1597.500   
           21    197.68    75.0  2021     11   30            1  14826.000   
           22     49.99   203.3  2021     11   30            1  10162.967   
           23     49.95   256.4  2021     11   30            1  12807.180   

                      cat  
date       hour            
2016-01-01 0          off  
           1          off  
           2          off  
           3          off  
           4          off  
...                   ...  
2021-11-30 19     evening  
           20    standard  
           21    standard  
           22         off  
           23         off  

[51172 rows x 8 columns]

In [5]:
flow=dat
#flow['date']=pd.to_datetime(flow['flowdate'],format="%d/%m/%Y") #"%Y-%m-%d")#"%d/%m/%Y")
#flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek

flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=dam['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.dropna(0)
flow=flow.reset_index()

flow

,date,hour,flow,from,to,year,month,day,day_of_week,price,value
0,2016-01-01,0,160,ZAMZ,ZAMC,2016,1,1,4,40.10,6416.00
1,2016-01-01,0,170,ZIM,ZAMZ,2016,1,1,4,40.10,6817.00
2,2016-01-01,1,160,ZAMZ,ZAMC,2016,1,1,4,40.08,6412.80
3,2016-01-01,1,170,ZIM,ZAMZ,2016,1,1,4,40.08,6813.60
4,2016-01-01,2,160,ZAMZ,ZAMC,2016,1,1,4,40.06,6409.60
...,...,...,...,...,...,...,...,...,...,...,...
370527,2021-11-30,23,10,RSAN,RSAS,2021,11,30,1,49.95,499.50
370528,2021-11-30,23,10,RSAS,NAM,2021,11,30,1,49.95,499.50
370529,2021-11-30,23,103,ZAMZ,ZAMC,2021,11,30,1,49.95,5144.85
370530,2021-11-30,23,103,ZIM,ZAMZ,2021,11,30,1,49.95,5144.85


In [6]:
f_to=flow[['date','hour','flow','value','price','to','year','month','day','day_of_week']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','price','from','year','month','day','day_of_week']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,price,node,year,month,day,day_of_week
0,2016-01-01,0,160,6416.00,40.10,ZAMC,2016,1,1,4
1,2016-01-01,0,170,6817.00,40.10,ZAMZ,2016,1,1,4
2,2016-01-01,1,160,6412.80,40.08,ZAMC,2016,1,1,4
3,2016-01-01,1,170,6813.60,40.08,ZAMZ,2016,1,1,4
4,2016-01-01,2,160,6409.60,40.06,ZAMC,2016,1,1,4
...,...,...,...,...,...,...,...,...,...,...
370527,2021-11-30,23,-10,-499.50,49.95,RSAN,2021,11,30,1
370528,2021-11-30,23,-10,-499.50,49.95,RSAS,2021,11,30,1
370529,2021-11-30,23,-103,-5144.85,49.95,ZAMZ,2021,11,30,1
370530,2021-11-30,23,-103,-5144.85,49.95,ZIM,2021,11,30,1


In [7]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='HCB'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [8]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows=flows.set_index(['date','hour'])
flows['cat']=dam['cat']
flows.reset_index()
flows

node  year  month  day  day_of_week    index  flow    value  \
date       hour                                                                
2016-01-01 0     ZAMC  2016      1    1            4        0   160  6416.00   
           0     ZAMZ  2016      1    1            4        1    10   401.00   
           0      ZIM  2016      1    1            4        1  -170 -6817.00   
           1     ZAMC  2016      1    1            4        2   160  6412.80   
           1     ZAMZ  2016      1    1            4        5    10   400.80   
...               ...   ...    ...  ...          ...      ...   ...      ...   
2021-11-30 23     LES  2021     11   30            1   370523    -5  -249.75   
           23     MOZ  2021     11   30            1   741049   -65 -3246.75   
           23     NAM  2021     11   30            1   370528    10   499.50   
           23     RSA  2021     11   30            1  2223156   -43 -2147.85   
           23    ZAMC  2021     11   30            1   370529   103  5144.85   

                  price  cat  
date       hour               
2016-01-01 0      40.10  off  
           0      80.20  off  
           0      40.10  off  
           1      40.08  off  
           1      80.16  off  
...                 ...  ...  
2021-11-30 23     49.95  off  
           23     99.90  off  
           23     49.95  off  
           23    299.70  off  
           23     49.95  off  

[213987 rows x 10 columns]

In [9]:
flowsb=flows.copy()
flowsb.loc[flowsb['flow']<0,'side']='supply'
flowsb.loc[flowsb['flow']>0,'side']='consume'


flow_monthly_cat=flowsb.groupby(['year','month', 'node','cat','side']).sum()
flow_monthly_cat.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly_cat['hour_count']=flowsb.groupby(['year','month', 'node','cat','side']).count()['flow']
flow_monthly_cat['price_weight_mean'] = flow_monthly_cat['value'] / flow_monthly_cat['flow_sum']
flow_monthly_cat['price_mean']=flowsb.groupby(['year','month', 'node','cat','side']).mean()['price']
flow_monthly_cat=flow_monthly_cat.drop(['day','day_of_week','index','price'],axis=1)
flow_monthly_cat=flow_monthly_cat.loc[flow_monthly_cat['flow_sum']!=0]
flow_monthly_cat['power']=flow_monthly_cat['flow_sum']/flow_monthly_cat['hour_count']
flow_monthly_cat

flow_sum      value  hour_count  \
year month node cat      side                                       
2016 1     BOT  evening  supply        -25   -1501.40           2   
                morning  supply       -285  -14572.40           4   
                off      supply      -1270  -59474.14          45   
                standard supply      -1165  -57809.40          18   
           HCB  evening  supply        -57   -6204.59           2   
...                                    ...        ...         ...   
2021 11    ZAMZ standard supply       -973 -162901.52          26   
           ZIM  evening  consume         1     301.37           1   
                off      supply       -455  -18474.43           8   
                standard consume         2     395.19           2   
                         supply      -4943 -431445.74          67   

                                  price_weight_mean   price_mean      power  
year month node cat      side                                                
2016 1     BOT  evening  supply           60.056000    60.050000 -12.500000  
                morning  supply           51.131228    55.290000 -71.250000  
                off      supply           46.830031    48.463556 -28.222222  
                standard supply           49.621803    56.768333 -64.722222  
           HCB  evening  supply          108.852456    99.140000 -28.500000  
...                                             ...          ...        ...  
2021 11    ZAMZ standard supply          167.421912   394.903846 -37.423077  
           ZIM  evening  consume         301.370000  1506.850000   1.000000  
                off      supply           40.603143   192.043750 -56.875000  
                standard consume         197.595000   987.975000   1.000000  
                         supply           87.284188   434.986567 -73.776119  

[2603 rows x 6 columns]

In [10]:
flow_monthly_cat.to_csv(out + pref + '_flow_monthly_cat.csv')